In [15]:
import pymysql
import pandas as pd
import random


In [10]:
db = pymysql.connect(user='root', password='1234', host='127.0.0.1', db='Movie_DB', cursorclass=pymysql.cursors.DictCursor)

cursor = db.cursor()

query = "select * from movie_db.영화"
cursor.execute(query)
db_export = cursor.fetchall()

# dataframe 입력 (불러오기)
db_movie_df=pd.DataFrame(db_export)
db_movie_df

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,내 여동생과 섹스해 줘,None,2024.0,한국,장편,성인물(에로),개봉예정,None,(주)샤이커뮤니케이션즈
1,불륜에 중독된 변태 유부녀,None,2024.0,한국,장편,성인물(에로),개봉예정,None,(주)샤이커뮤니케이션즈
2,남동생마저 노리는 욕정녀,None,2024.0,한국,장편,성인물(에로),개봉예정,None,(주)샤이커뮤니케이션즈
3,성욕이 너무 강한 글래머,None,2024.0,한국,장편,성인물(에로),개봉예정,None,(주)샤이커뮤니케이션즈
4,처음 본 대물에 젖어버린 유부녀,None,2024.0,한국,장편,성인물(에로),개봉예정,None,(주)샤이커뮤니케이션즈
...,...,...,...,...,...,...,...,...,...
65523,최종면접,Last Interview,2012.0,한국,단편,드라마,기타,현문섭,None
65524,플라멩코 소녀,Flamenco Nina,2013.0,한국,단편,드라마,기타,이찬호,None
65525,KTX 특별열차,KTX Extra,2009.0,한국,단편,드라마,기타,정수진,None
65526,무더운 하루,None,2013.0,한국,단편,기타,기타,None,None


In [11]:
query = "select * from movie_db.감독"
cursor.execute(query)
db_export = cursor.fetchall()

# dataframe 입력 (불러오기)
db_director_df=pd.DataFrame(db_export)
db_director_df

,peopleCd,peopleNm,peopleNmEn,repRoleNm,filmoNames
0,20396007,이세영,None,감독,나는 아빠다|대한민국 1%
1,20331280,이용남,None,감독,행복의 발견|유 돈 노우|장마|느루
2,20395966,기시 요시유키,Yoshiyuki Kishi,감독,정욕
3,20395965,니콜라우스 게이할터,Nikolaus Geyrhalter,감독,아름다운 쓰레기 여행
4,20395964,키무라 별 르무안,KIMURA byol LEMOINE,감독,입양아에서 입양인으로: 한국 국제 입양의 70년
...,...,...,...,...,...
24927,10090862,휴버트 프랭크,None,감독,카프리의 깊은밤 3
24928,10090871,흐라픈 군라프슨,Hrafn Gunnlaugsson,감독,사랑이 눈뜰 때
24929,10090882,히가시 요이치,Yoichi Higashi,감독,"러브 호텔 무삭제판|술이 깨면 집에 가자|간호사, 나츠코의 뜨거운 여름|나의 조교일..."
24930,10090896,히구친스키,Higuchinsky,감독,소용돌이


In [12]:
query = "select * from movie_db.배우"
cursor.execute(query)
db_export = cursor.fetchall()

# dataframe 입력 (불러오기)
db_actor_df=pd.DataFrame(db_export)
db_actor_df

,peopleCd,peopleNm,peopleNmEn,repRoleNm,filmoNames
0,20395989,김정연,None,배우,착한 아이 사탕이|제6회 김포국제청소년영화제\t 폐막작
1,20392256,김수진,None,배우,달짝지근해: 7510|브릿지
2,20395987,아디나 바잔,Adina BAZHAN,배우,다우렌의 결혼
3,20331052,이은미,LEE Eun-mi,배우,거짓말의 색은 노랑|블라인드|보속|내 동생|시선|타래 |목격담|김다육
4,20395962,정다민,JUNG Da-min,배우,수학영재 형주
...,...,...,...,...,...
47668,10090907,히다카 노리코,Noriko Hidaka,배우,극장판 이누야샤 4 - 홍련의 봉래도|이웃집 토토로
47669,10090917,히데오 모리,None,배우,요코하마 메리
47670,10090980,히로시 시미즈,Hiroshi Shimizu,배우,포스트맨 블루스
47671,10091039,히토토 요,Yo Hitoto,배우,카페 뤼미에르


In [29]:
# 고유한 장르 목록을 확인하는 코드
def check_genre():
    unique_genres = db_movie_df['장르'].unique()
    genreList = []
    for genre in unique_genres:
        try:
            x = genre.split(',')
            genreList.extend(x)
        except AttributeError:
            pass

    genreList=list(set(genreList)) # 장르의 종류
    genreList.remove('기타')
    return genreList

In [28]:
# 장르별 영화추천
def recommend_movie(genre,df):

    # 장르가 '드라마'인 영화 선택
    drama_movies = df[df['장르'] == genre]['영화명'].tolist()

    # 장르가 없는 영화인 경우
    if not drama_movies:
        return None
    else:
        # 랜덤으로 추천
        recommended_movie = random.choice(drama_movies)
        return recommended_movie

# 함수 호출하여 추천 받기
movie = recommend_movie('드라마',db_movie_df)

# 결과 출력
if recommended_movie:
    print("추천 영화:", movie)
else:
    print("장르가 '드라마'인 영화가 없습니다.")

db_movie_df[db_movie_df['영화명'] == movie]

추천 영화: 처제지만 괜찮아 무삭제


,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
26426,처제지만 괜찮아 무삭제,None,2021.0,한국,장편,드라마,기타,최성은,(주)영화사히트
